In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from hyperopt import hp, tpe, fmin, Trials
from tqdm import tqdm

import datetime
import copy
import json

import sys
sys.path.append('../../')

from data.dataloader import Covid19IndiaLoader
from data.processing import get_data
from data.processing import granular, processing

from models.seir import SEIR_Testing, SEIRHD, SEIR_Movement, SEIR_Movement_Testing, SEIRHD_Severity, SEIRHD_Bed

from main.seir.fitting import train_val_split, single_fitting_cycle, get_variable_param_ranges
from main.seir.forecast import get_forecast, create_region_csv, create_all_csvs, write_csv
from main.seir.optimiser import Optimiser


from utils.create_report import create_report
from utils.loss import Loss_Calculator
from utils.smooth_jump import smooth_big_jump_stratified, smooth_big_jump
from viz import plot_forecast, plot_fit

## Load Covid19india Data

In [ ]:
loader = Covid19IndiaLoader()
dataframes = loader.get_covid19india_api_data()

In [ ]:
predictions_dict = {}

## Select Districts to fit on

In [ ]:
districts_to_show = [('Maharashtra', 'Mumbai')]

## Get Variable Params

In [ ]:
variable_param_ranges = {
    'lockdown_R0': (1, 1.5),
    'T_inc': (4, 5),
    'T_inf': (3, 4),
    'T_recov_hq': (50, 70),
    'T_recov_non_o2_beds': (20, 30),
    'T_recov_o2_beds': (35, 50),
    'T_recov_icu': (20, 40),
    'T_recov_ventilator': (50, 60),
    'P_non_oxy': (0, 0.15),
    'P_oxy': (0, 0.15),
    'P_icu': (0, 0.05),
    'P_vent': (0, 0.05),
    'P_fatal': (0, 0.2),
    'E_hosp_ratio': (0, 2),
    'I_hosp_ratio': (0, 1)
}
variable_param_ranges = get_variable_param_ranges(variable_param_ranges)

## Assign Filename var

In [ ]:
filename = '../../data/data/mumbai/case_summary_expanded_2006.csv'

## Perform M1 and M2 fits

In [ ]:
for state, district in districts_to_show:
    predictions_dict[(state, district)] = {}
    predictions_dict[(state, district)]['m1'] = single_fitting_cycle(
        dataframes, state, district, data_from_tracker=False, granular_data=True, filename=filename, #Data
        model=SEIRHD_Bed, variable_param_ranges=variable_param_ranges, #Choose Model and Ranges
        train_period=7, val_period=7, num_evals=1000, initialisation='intermediate', #Optimisation related parameters
        which_compartments=['hospitalised', 'deceased', 'hq', 'non_o2_beds', 'o2_beds', 'icu', 'ventilator'], #Compartments to Apply Loss on 
        smooth_jump=True, smoothing_length=33, smoothing_method='weighted') #Smoothing
    
    predictions_dict[(state, district)]['state'] = state
    predictions_dict[(state, district)]['dist'] = district
    predictions_dict[(state, district)]['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
    predictions_dict[(state, district)]['datasource'] = 'covid19api' if predictions_dict[(state, district)]['m1']['data_from_tracker'] else 'municipality'
    predictions_dict[(state, district)]['variable_param_ranges'] = predictions_dict[(state, district)]['m1']['variable_param_ranges']
#     predictions_dict[(state, district)]['data_last_date'] = predictions_dict[(state, district)]['m2']['data_last_date']

In [ ]:
predictions_dict[('Maharashtra', 'Mumbai')]['m1']['df_loss']

In [ ]:
predictions_dict[('Maharashtra', 'Mumbai')]['m1']['best_params']

## Gridsearch + Bayesian Opt

#### Setup

In [ ]:
train_period = 7
val_period = 7
loss_indices = [-train_period, None]
model=SEIRHD
state = 'Maharashtra'
district = 'Mumbai'
which_compartments = ['hospitalised', 'deceased', 'hq', 'non_o2_beds', 'o2_beds', 'icu', 'ventilator']

#### Load Data and Smooth

In [ ]:
df_district = granular.get_data(filename)
orig_df_district = copy.copy(df_district)
df_district = smooth_big_jump_stratified(df_district)

#### Create train-val-split

In [ ]:
df_train, df_val = train_val_split(df_district, train_rollingmean=True, val_rollingmean=True, val_size=val_period)
df_train_nora, df_val_nora = train_val_split(df_district, train_rollingmean=False, val_rollingmean=False, val_size=val_period)

#### Fit

In [ ]:
comp_name = 'ventilator'
var_name = 'T_recov_{}'.format(comp_name)
# var_name = 'lockdown_R0'
variable_param_ranges = {
    var_name: np.linspace(1, 100, 201)
}

In [ ]:
default_params = copy.copy(predictions_dict[('Maharashtra', 'Mumbai')]['m1']['best_params'])
try:
    for key in variable_param_ranges.keys():
        del default_params[key]
except Exception as err:
    print('')
optimiser = Optimiser()
extra_params = optimiser.init_default_params(df_train, N=1e7, initialisation='intermediate', train_period=train_period)
default_params = {**default_params, **extra_params}
total_days = (df_train.iloc[-1, :]['date'] - default_params['starting_date']).days
loss_array, params_dict = optimiser.gridsearch(df_train, default_params, variable_param_ranges, model=model, method='mape', 
                                               loss_indices=loss_indices, which_compartments=[comp_name, 'recovered'], total_days=total_days, debug=False)
best_params = params_dict[np.argmin(loss_array)]
df_prediction = optimiser.solve(best_params, default_params, df_train, end_date=df_val.iloc[-1, :]['date'], model=model)

plt.figure(figsize=(12, 12))
plt.plot([x[list(x.keys())[0]] for x in params_dict], loss_array)
plt.ylabel('Loss Value (MAPE)')
plt.xlabel('{}'.format(var_name))
plt.title('How Loss for compartment {} changes as {} changes'.format(comp_name, var_name))
plt.grid()

In [ ]:
df_prediction = optimiser.solve({'T_recov_icu': 100}, default_params, df_train, end_date=df_train.iloc[-1, :]['date'], model=model)
lc = Loss_Calculator()
lc._calc_mape(df_prediction['icu'], df_train['icu'].iloc[-train_period:].reset_index(drop=True)) 

In [ ]:
params_dict

In [ ]:
df_val['hospitalised'].diff(), df_val['hq'].diff(), df_val['non_o2_beds'].diff(), df_val['o2_beds'].diff(), df_val['icu'].diff(), df_val['ventilator'].diff()

In [ ]:
fig = plot_fit(df_prediction, df_train, df_val, df_train_nora, df_val_nora, train_period, state, district, which_compartments=which_compartments)
plt.show()

#### Plot

In [ ]:
which_compartments = ['hospitalised', 'deceased', 'hq', 'non_o2_beds', 'o2_beds', 'icu', 'ventilator']
plot_fit(df_prediction, df_train, df_val, df_train_nora, df_val_nora, train_period, state, district,
         which_compartments=which_compartments)

## Create Master Loss Dataframe

### M1 Loss DataFrame

In [ ]:
lc = Loss_Calculator()

In [ ]:
lc.create_loss_dataframe_master(predictions_dict, 'm1')

### M2 Loss DataFrame

In [ ]:
lc.create_loss_dataframe_master(predictions_dict, 'm2')

## Plot Forecasts

In [ ]:
for region in predictions_dict.keys():
    predictions_dict[region]['forecast'] = plot_forecast(predictions_dict[region], region, both_forecasts=True, error_bars=True)

## Create Report

In [ ]:
for region in predictions_dict.keys():
    create_report(predictions_dict[region])

## Create and Save Output CSV

In [ ]:
df_output = create_all_csvs(predictions_dict, icu_fraction=0.02)

In [ ]:
write_csv(df_output)